In [18]:
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy.stats import zscore
from src.utils.data import getDataPandas

In [19]:
data = getDataPandas()
data = data.sample(frac=1, random_state=1)
data = data.drop_duplicates(subset=['PATNO', 'EVENT_ID'], keep='first').reset_index(drop=True)

In [20]:
data = data.drop(data[data['NUPDR3OF'] < 5].index).reset_index(drop=True)

In [21]:
def load_img(rec):
    img_data = np.array(nib.load(rec.T1_MNI_PATH).get_fdata())
    return img_data

In [22]:
data['T1'] = data.apply(load_img, axis=1)

In [23]:
vox = np.array([np.array(l) for l in data['T1']])
vox = np.reshape(vox, (data.shape[0], -1))
vox = zscore(vox, axis=1)

In [24]:
x = data[['NUPDR3ON', 'AGE_AT_VISIT', 'SEX', 'DURATION']]
y = data[['CAT']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12, stratify=y)

In [25]:
print(len(y_train[y_train['CAT']==1]))
print(len(y_train[y_train['CAT']==0]))
print(len(y_test[y_test['CAT']==1]))
print(len(y_test[y_test['CAT']==0]))

94
94
24
24


In [26]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.9)
train_features = pca.fit_transform(vox[x_train.index])
train_fe = pd.DataFrame(train_features)
test_features = pca.transform(vox[x_test.index])
test_fe = pd.DataFrame(test_features)
x_train = x_train.reset_index(drop=True)
x_train = x_train.join(train_fe)
#x_train[['NUPDR3ON', 'AGE_AT_VISIT', 'DURATION']] = x_train[['NUPDR3ON', 'AGE_AT_VISIT', 'DURATION']].apply(zscore)
x_test = x_test.reset_index(drop=True)
x_test = x_test.join(test_fe)
#x_test[['NUPDR3ON', 'AGE_AT_VISIT', 'DURATION']] = x_test[['NUPDR3ON', 'AGE_AT_VISIT', 'DURATION']].apply(zscore)

In [27]:
from scipy.stats import ttest_ind, chi2_contingency, normaltest, ranksums
print(normaltest(x_train['NUPDR3ON']))
print(normaltest(x_train['AGE_AT_VISIT']))
print(normaltest(x_train['DURATION']))
print()
print(normaltest(x_test['NUPDR3ON']))
print(normaltest(x_test['AGE_AT_VISIT']))
print(normaltest(x_test['DURATION']))
print()
print(ranksums(x_train['NUPDR3ON'], x_test['NUPDR3ON']))
print(ttest_ind(x_train['AGE_AT_VISIT'], x_test['AGE_AT_VISIT']))
print(ranksums(x_train['DURATION'], x_test['DURATION']))
_, p, _, _ = chi2_contingency([[len(x_test[x_test['SEX']==0]), len(x_train[x_train['SEX']==0])], [len(x_test[x_test['SEX']==1]), len(x_train[x_train['SEX']==1])]])
print(p)
_, p, _, _ = chi2_contingency([[len(y_test[y_test['CAT']==0]), len(y_train[y_train['CAT']==0])], [len(y_test[y_test['CAT']==1]), len(y_train[y_train['CAT']==1])]])
print(p)

NormaltestResult(statistic=22.655799171622807, pvalue=1.2032495788117283e-05)
NormaltestResult(statistic=4.852198204589301, pvalue=0.08838092596077546)
NormaltestResult(statistic=7.240945037015648, pvalue=0.026770024171769644)

NormaltestResult(statistic=11.733850675659243, pvalue=0.0028315660726364014)
NormaltestResult(statistic=1.5265434127925936, pvalue=0.46613886000729177)
NormaltestResult(statistic=16.560930298787518, pvalue=0.00025341929478374267)

RanksumsResult(statistic=-0.27951100645439675, pvalue=0.7798526925597085)
Ttest_indResult(statistic=-1.0419742684085629, pvalue=0.29849893355701596)
RanksumsResult(statistic=0.6667995620077346, pvalue=0.5049001725519073)
0.4902672148591981
1.0


In [28]:
x1 = x[y['CAT']==1]
x0 = x[y['CAT']==0]
print(normaltest(x1['NUPDR3ON']))
print(normaltest(x1['AGE_AT_VISIT']))
print(normaltest(x1['DURATION']))
print()
print(normaltest(x0['NUPDR3ON']))
print(normaltest(x0['AGE_AT_VISIT']))
print(normaltest(x0['DURATION']))
print()
print(ranksums(x0['NUPDR3ON'], x1['NUPDR3ON']))
print(ranksums(x0['AGE_AT_VISIT'], x1['AGE_AT_VISIT']))
print(ranksums(x0['DURATION'], x1['DURATION']))
_, p, _, _ = chi2_contingency([[len(x1[x1['SEX']==0]), len(x0[x0['SEX']==0])], [len(x1[x1['SEX']==1]), len(x0[x0['SEX']==1])]])
print(p)

NormaltestResult(statistic=6.651504515461584, pvalue=0.03594546832382435)
NormaltestResult(statistic=2.193381412613865, pvalue=0.3339744765945661)
NormaltestResult(statistic=4.07495714050031, pvalue=0.13035698282177208)

NormaltestResult(statistic=15.13836978400176, pvalue=0.0005161129650122217)
NormaltestResult(statistic=6.188144537159755, pvalue=0.045317035001662696)
NormaltestResult(statistic=5.933532013824612, pvalue=0.0514694934556066)

RanksumsResult(statistic=7.249189246390306, pvalue=4.192737282670555e-13)
RanksumsResult(statistic=2.2015491213882963, pvalue=0.02769717324567565)
RanksumsResult(statistic=-3.914924509493436, pvalue=9.043249780012692e-05)
1.0


In [29]:
model = XGBClassifier()
parameters = {'nthread': [4],
              'objective': ['binary:logistic'],
              'learning_rate': [0.2, 0.3],
              'max_depth': [1, 3, 5],
              'min_child_weight': [1, 3, 5],
              'subsample': [0.8, 0.85, 0.9],
              'colsample_bytree': [0.75, 0.8, 0.85],
              'n_estimators': [50, 100, 200],
              'missing': [-999],
              'random_state': [1]}
clf = GridSearchCV(model, parameters,
                        n_jobs=5,
                        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=1),
                        verbose=3,
                        #n_iter=200,
                        scoring='roc_auc')

In [30]:
clf.fit(x_train, y_train)

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=5,
             param_grid={'colsample_bytree': [0.75, 0.8, 0.85],
                         'learning_rate': [0.2, 0.3], 'max_depth': [1, 3, 5],
                         'min_child_weight': [1, 3, 5], 'missing': [-999],
                         'n_estimators': [50, 100, 200], 'nthread': [4],
                         'objective': ['binary:logistic'], 'random_state': [1],
                         'subsample': [0.8, 0.85, 0.9]},
             scoring='roc_auc', verbose=3)

In [31]:
clf.best_params_

{'colsample_bytree': 0.75,
 'learning_rate': 0.3,
 'max_depth': 3,
 'min_child_weight': 3,
 'missing': -999,
 'n_estimators': 100,
 'nthread': 4,
 'objective': 'binary:logistic',
 'random_state': 1,
 'subsample': 0.9}

In [32]:
y_prob = clf.best_estimator_.predict_proba(x_test)
print('AUC train {}, test {}'.format(clf.best_score_, metrics.roc_auc_score(list(y_test['CAT']), y_prob[:, 1])))

AUC train 0.7285626346568176, test 0.7170138888888888


In [33]:
y_pred = clf.best_estimator_.predict(x_train)
print(metrics.classification_report(list(y_train['CAT']), y_pred))
print(metrics.confusion_matrix(list(y_train['CAT']), y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00        94

    accuracy                           1.00       188
   macro avg       1.00      1.00      1.00       188
weighted avg       1.00      1.00      1.00       188

[[94  0]
 [ 0 94]]


In [34]:
y_pred = clf.best_estimator_.predict(x_test)
print(metrics.classification_report(list(y_test['CAT']), y_pred))
print(metrics.confusion_matrix(list(y_test['CAT']), y_pred))

              precision    recall  f1-score   support

           0       0.59      0.67      0.63        24
           1       0.62      0.54      0.58        24

    accuracy                           0.60        48
   macro avg       0.61      0.60      0.60        48
weighted avg       0.61      0.60      0.60        48

[[16  8]
 [11 13]]
